<a href="https://colab.research.google.com/github/Bart-del/studies-machine-learning/blob/main/Zadanie_4_Bart%C5%82omiej_Wysocki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Zadanie 4. Regresja liniowa wielu zmiennych

Stwórz model regresji liniowej dla cen mieszkań w Poznaniu. Zestaw danych to 5000 ogłoszeń z serwisu gratka.pl.

Miarą błędu na danych testowych niech będzie średnia kwadratowa błędu (Root Mean Squared Error, RMSE) dla cen mieszkań.

Plik gratkapl.csv zawiera dane, na których należy zbudować model regresyjny. Pierwszy wiersz w pliku to nagłówek zawierający opisy kolumn:

    Id - numer porządkowy oferty
    Expected - cena mieszkania
    Rooms - liczba pokojów
    SqrMeters - metraż mieszkania w m2
    Floor - piętro
    Location - dzielnica lub adres w Poznaniu
    Description - fragment opisu mieszkania

In [7]:
from google.colab import files
uploaded = files.upload()



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Saving gratkapl.csv to gratkapl.csv


<h1>Wczytanie danych 

In [8]:
import csv
reader = csv.reader(open('gratkapl.csv', encoding='utf-8'), delimiter=',')
data = pd.DataFrame(reader)
new_header = data.iloc[0]
data = data[1:]
data.columns = new_header

<h1>Podzielenie danych na zbiór uczący i testowy

In [9]:
del data["Location"]
del data["Description"]
del data["Id"]

data['Expected'] = pd.to_numeric(data['Expected'])
data['Rooms'] = pd.to_numeric(data['Rooms'])
data['Floor'] = pd.to_numeric(data['Floor'])
data['SqrMeters'] = pd.to_numeric(data['SqrMeters'])

trening_set = data[:4000]
test_set = data[4000:]

<h1>Usuwanie wartości odstających

In [10]:
from scipy import stats
              
trening_set = trening_set[(np.abs(stats.zscore(trening_set)) < 3).all(axis=1)]
test_set = test_set[(np.abs(stats.zscore(test_set)) < 3).all(axis=1)]

<h1>Hipoteza

In [11]:
def hMx(theta, X):
    return X * theta

<h1>Funkcja kosztu


In [16]:
def JMx(theta,X,y):
    """Wersja macierzowa funkcji kosztu"""
    m = len(y)
    J = 1.0 / (2.0 * m) * ((X * theta - y) . T * ( X * theta - y))
    return J.item()


In [12]:
def dJMx(theta,X,y):
    return 1.0 / len(y) * (X.T * (X * theta - y)) 

In [13]:
m, n_plus_1 = trening_set.values.shape
n = n_plus_1 - 1
Xn = trening_set.values[:, 1:].reshape(m, n)

# Dodaj kolumnę jedynek do macierzy
XMx = np.matrix(np.concatenate((np.ones((m, 1)), Xn), axis=1)).reshape(m, n_plus_1)
yMx = np.matrix(trening_set.values[:, 0]).reshape(m, 1)


<h1>Metoda gradientu prostego

In [14]:
def GDMx(fJ, fdJ, theta, X, y, alpha, eps):
    current_cost = fJ(theta, X, y)
    log = [[current_cost, theta]]
    while True:
        theta = theta - alpha * fdJ(theta, X, y) # implementacja wzoru
        current_cost, prev_cost = fJ(theta, X, y), current_cost
        if abs(prev_cost - current_cost) <= eps:
            break
        if current_cost > prev_cost:
            print('Długość kroku (alpha) jest zbyt duża!')
            break
        log.append([current_cost, theta])
    return theta, log

In [17]:
thetaStartMx = np.zeros((n + 1, 1))

# Zmieniamy wartości alpha (rozmiar kroku) oraz eps (kryterium stopu)
thetaBestMx, log = GDMx(JMx, dJMx, thetaStartMx, 
                        XMx, yMx, alpha=0.0001, eps=0.1)

<h1>Ewaluacja modelu

In [18]:
def mse(expected, predicted):
    """Błąd średniokwadratowy"""
    m = len(expected)
    if len(predicted) != m:
        raise Exception('Wektory mają różne długości!')
    return 1.0 / (2 * m) * sum((expected[i] - predicted[i])**2 for i in range(m))



In [23]:
j, i_plus_1 = test_set.values.shape
i = i_plus_1 - 1
Xn1 = test_set.values[:, 1:].reshape(j, i)
XMx1 = np.matrix(np.concatenate((np.ones((j, 1)), Xn1), axis=1)).reshape(j, i_plus_1)
yMx1 = np.matrix(test_set.values[:, 0]).reshape(j, 1)

y_pred = hMx(thetaBestMx, XMx1)

evaluation_result = mse(yMx1, y_pred)

In [24]:
print(evaluation_result)

[[3.76410227e+09]]
